In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.widgets import Slider
from IPython.display import display
import glob
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [15]:
# PCA over all the frames - flattened

# importing all the data, getting all the frames

TRAINING_DATA_PATH = "../data/training_data/"
files = sorted(glob.glob(TRAINING_DATA_PATH + "mixed_samples/*.npy"))

heart_samples = []

for f in files:
    data = np.load(f)
    heart_samples.append(data)
heart_samples = np.array(heart_samples)
 
print(heart_samples.shape)

heart_xyz = heart_samples[..., :3]
X = heart_xyz.reshape(len(heart_xyz), -1)
print(X.shape)

# Importing demographic data
demographics = pd.read_csv(TRAINING_DATA_PATH + "mixed_demographics.csv")

# Doing PCA
pca = PCA(n_components = 50)
X_pca = pca.fit_transform(X)
print(pca.explained_variance_ratio_)
print(X_pca.shape)

(900, 10, 18000, 4)
(900, 540000)
[0.30516536 0.2456048  0.09653941 0.04462171 0.02640237 0.01987551
 0.01842039 0.01475463 0.01414272 0.01354737 0.0124781  0.01121955
 0.01086073 0.00971639 0.00848699 0.00800505 0.00715335 0.00680103
 0.00578201 0.00484285 0.00412508 0.00342057 0.00253416 0.00245202
 0.00238894 0.00235305 0.0023391  0.00232305 0.00231017 0.0022946
 0.0022599  0.00222634 0.00221541 0.00210907 0.00210121 0.00203296
 0.00201278 0.00197192 0.00189014 0.00183691 0.0017997  0.00175722
 0.00171487 0.00169146 0.00167237 0.00162767 0.00160665 0.00158726
 0.001535   0.0014912 ]
(900, 50)


In [29]:
# plots for PCA - simular to Cayla's done


In [38]:
# linear regression on point cloud data
y = demographics["MI"]

X_train, X_test, y_train, y_test = train_test_split(X_pca[:, :3], y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)  # fit on train only
X_test_std  = scaler.transform(X_test)       # transform test with same params

std_logreg = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000))
])
std_logreg.fit(X_train, y_train)

# Evaluate
y_pred = std_logreg.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

     healthy       0.52      0.60      0.56        90
         pMI       0.53      0.44      0.48        90

    accuracy                           0.52       180
   macro avg       0.52      0.52      0.52       180
weighted avg       0.52      0.52      0.52       180



In [ ]:
# K-means clusting to try and figure out hearts that aren't anatompically possible
